# Minesweeper with propositional logic

## Goal of the project
The goal of the project is to implement an **intelligent agent** capable of autonomously playing the *Minesweeper* game using ***propositional logic***. The code base used is the one proposed by Harvard, available at [this link](https://cdn.cs50.net/ai/2020/spring/projects/1/minesweeper.zip).  
The structure of the application, including the user interface (UI), was already provided, however, the implementation of the agent was done entirely by me.

## Introduction to Minesweeper
Minesweeper is a single-player logic game whose goal is to discover all the cells in a grid that do not contain mines.

### Minesweeper Features:
- **Deterministic**: The player's actions do not affect the outcome of the game randomly. Every action has a predictable outcome.
- **Fully Observable**: Once a cell is discovered, the information about that cell is completely visible to the player.
- **Game Environment**: A grid* of hidden cells, some of which contain mines**, each discovered cell shows the number of adjacent mines.
    - '*' 6x6.
    - '**' in total there are 6 mines. 
  

## Propositional Logic
Propositional logic is a branch of logic that deals with *propositions* and their combinations through *logical connectives* such as **AND, OR, NOT**, etc. Through this logic it is possible to represent and manipulate knowledge.

### Components of Propositional Logic


**Propositions**

 Propositions are statements that can be **true** or **false**.
 - Examples of propositions: "Cell (1,1) is a mine".
 - Propositions are usually denoted by capital letters, for example,:\( C_1_1 \).
 
 **Logical Connectives**
 
 Logical connectives are operators used to combine propositions and form more complex logical expressions.
 - **AND (∧)**: The conjunction of two propositions is true only if both propositions are true.
 - **OR (∨)**: The disjunction of two propositions is true if at least one of the propositions is true.
 - **NOT (¬)**: The negation of a proposition is true if the proposition is false.
 - **IMPLIES (→)**: The implication between two propositions is false only if the first proposition is true and the second is false.
 - **IFF (↔)**: The logical equivalence between two propositions is true if both propositions are both true or both false.

**Inference Rules**

In propositional logic, through rules, such as modusponens or inference algorithms, such as **truth tables**, **forward chaining** and **backward chaining**, it is possible to deduce new propositions starting from existing propositions.



## `Cell` class
The `Cell` class represents a cell within the Minesweeper game, with information about its location, the presence of a mine, and the number of adjacent mines.

**Important note**: in addition to the various `set` and `get` methods, it was necessary to override the functions:

- `__eq__(self, other)`: To compare whether two cells are equal based on their position.
- `__hash__(self)`: to make the cell hashable, allowing it to be used as a key in a dictionary or as an element of a set.

In [28]:
class Cell:
    row = None
    col = None
    number = None
    has_mine = False
    def __init__(self,row=None,col=None):
        self.row = row
        self.col = col
    def set_has_mine(self,value):
        self.has_mine = value
    def set_number(self,n):
        self.number=n
    def get_has_mine(self):
        return self.has_mine
    def get_row(self):
        return self.row
    def get_col(self):
        return self.col
    def get_number(self):
        return self.number
    def __eq__(self, other):
        '''Override for self==other'''
        if isinstance(other, Cell):
            return self.row == other.row and self.col == other.col
        return False
    # Without the __hash__ function, comparing sets containing objects of the Cell class would not work correctly 
    # because the set comparison operation requires the elements of the set to be hashable. 
    # In Python, an object must implement the __hash__ method to be hashable, which means it can be used as a key in a dictionary or as an element in a set.
    def __hash__(self):
        return hash((self.row, self.col))

## `Minesweeper` class


The `Minesweeper` class manages the game i.e. the initialization of the field, the placement of mines and the calculation of the number of mines adjacent to each cell.

In [ ]:
import random
from cell import Cell
# It is a representation of the game
class Minesweeper():
    def __init__(self, height, width, mines):
        self.height = height
        self.width = width
        # This set represents the locations of the cells that have mines.
        # Is used, with the mines_found set, to check the victory
        self.mines = set()
        # No mines found
        self.mines_found = set()
        # Empty field
        self.field = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                row.append(Cell(i,j))
            self.field.append(row)
        # Add mines randomly
        count = 0
        while count != mines:
            i = random.randrange(self.height)
            j = random.randrange(self.width)
            #check each cell
            if not self.field[i][j].get_has_mine():
                self.mines.add(Cell(i, j))
                self.field[i][j].set_has_mine(True)
                count+=1

        
        # Added the number of nearby mines.
        self.compute_nearby_mines()
        self.print()

    def get_cell_from_field(self,cell):
        for i in range(self.height):
            for j in range(self.width):
                if self.field[i][j] == cell:
                    return self.field[i][j]
        return None

    def print(self):
        for i in range(self.height):
            print("--" * self.width + "-")
            for j in range(self.width):
                if self.field[i][j].get_has_mine():
                    print("|X", end="")
                else:
                    print(f"|{self.field[i][j].get_number()}", end="")
            print("|")
        print("--" * self.width + "-")
        string = "Mines are in: "
        for cell in self.mines:
            string+=f"({cell.get_row()},{cell.get_col()})"
        print(string)

    def is_mine(self,cell):
        return self.get_cell_from_field(cell).get_has_mine()

    def compute_nearby_mines(self):
        # Keep count of nearby mines
        directions = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1),         (0, 1),
            (1, -1), (1, 0), (1, 1)
        ]
        for row in range(len(self.field)):
            for col in range(len(self.field[0])):
                if self.field[row][col].get_has_mine():
                    continue
                mine_count = 0
                for dr, dc in directions:
                    r, c = row + dr, col + dc
                    if 0 <= r < len(self.field) and 0 <= c < len(self.field[0]):
                        if self.field[r][c].get_has_mine():
                            mine_count += 1
                self.field[row][col].set_number(mine_count)

    def get_nearby_mines(self, cell):
        '''Returns the number of nearby mines of a given cell'''
        return self.get_cell_from_field(cell).number


Parlare in ordine:
Sentence : Quando parlo della classe sentece spiego come faccio l'inferenza e perché non ho usato un algirtmo classico di inferenza tipo model checking a cuasa della complessità combinatoria del gioco e quindi ho usato la logica deduttiva .
Agent. 